### Import python packages 

In [1]:
import tweepy
import mysql.connector
from mysql.connector import Error
from textblob import TextBlob
import re

### Set predefined variables  

In [2]:
'''
store words to track in a list 
'''

word_s_tracked = ['#NSWfires', '#AustraliaOnFire', '#AustraliaisBurning', '#AustralianBushfire', '#AustalianBushfiresDisaster']


''' 
Store mysql table name and attributes information 
'''


table_name = 'aus_ameliorated_1'

tables_attributes = "id_str VARCHAR(255), text VARCHAR(255), polarity INT, subjectivity INT,\
                      created_at DATETIME, location VARCHAR(255),user_Description VARCHAR(255),is_user_verified BOOLEAN,\
                      user_followers INT, protected_tweets BOOLEAN, \
                      tweet_reply_count INT,retweet_count INT, favorite_count INT, latitude DOUBLE ,longitude DOUBLE"

### Twitter access credentials

In [3]:
# Consumer API keys

consumer_key = '................................ ' # Obtain API key from Twitter
consumer_secret_key = '..........................' # Obtain secret API key from Twitter 

# Access token & access token secret 

access_token = '................................' # obtain from twitter 
access_token_secret = '.........................'  # obtain from twitter 

### Tweet extraction , cleaning and insertion into Mysql database

In [4]:
''' Configure Twitter streaming api '''

class MyStreamListener(tweepy.StreamListener):
    
    
    def on_status(self, status):
        
        #Avoiding tweets that have being retweeted
        if not status.retweeted and ('RT @' not in status.text):    #this helps to extract only original tweets
            id_str = status.user.id
            text = deEmojify(status.text)
            text = clean_tweet(self, text) 
            sentiment = TextBlob(text).sentiment
            polarity = sentiment.polarity
            subjectivity = sentiment.subjectivity
            created_at = status.created_at
            location = deEmojify(status.user.location)
            user_Description = deEmojify(status.user.description)
            is_user_verified = status.user.verified  #to check if user is verified. 
            user_followers = status.user.followers_count # Number of users who follow this user
            protected_tweets = status.user.protected #Indicate if this user's Tweets are protected. 
            tweet_reply_count =  status.reply_count # Number of Replies of this Tweet.
            longitude = None
            latitude = None
            if status.coordinates:
                latitude = status.coordinates['coordinates'][0]  
                longitude = status.coordinates['coordinates'][1]     
            retweet_count = status.retweet_count
            favorite_count = status.favorite_count
             
            print(status.text)

        
            # Connect to mysql database 
        
            mysqldb = mysql.connector.connect(host ='localhost',
            database= 'twitterdb', # Note that , you must have created this db before hand in mysql
            user='root', 
            password='Foubizlee-260495',
            charset='utf8')
        
            if mysqldb.is_connected(): 
                '''
               check if table exist. If not, then create a new one.
                '''
                mycursor = mysqldb.cursor()
                checktablequery = """ 
                SELECT COUNT(*) 
                FROM information_schema.tables 
                WHERE table_name = '{0}'
                """.format(table_name)
                mycursor.execute(checktablequery)
            if mycursor.fetchone()[0] != 1:
                mycursor.execute(f"CREATE TABLE {table_name} ({tables_attributes })")
                mysqldb.commit()
            mycursor.close()  
    

            # Insert twitter data into mysql database
        
            if mysqldb.is_connected():
                mycursor = mysqldb.cursor() 
                query = f"INSERT INTO {table_name} (id_str, text, polarity, subjectivity, created_at,location,user_Description,is_user_verified,user_followers,protected_tweets,tweet_reply_count, retweet_count, favorite_count,latitude,longitude) VALUES (%s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                values = (id_str, text, polarity, subjectivity, created_at,location,user_Description,is_user_verified,\
                      user_followers,protected_tweets,tweet_reply_count,retweet_count, favorite_count,latitude,longitude)
                mycursor.execute(query, values)
                mysqldb.commit()
                mycursor.close()
        
        # Catch error and disconnect from stream
    def on_error(self, status_code):
        if status_code == 420:
        #returning False in on_data disconnects the stream
           return False

### Functions  used above for cleaning tweets

In [5]:
# Clean tweets to enable sentiment analysis 

def clean_tweet(self, tweet): 
    ''' 
    Use regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 


def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

### Twitter Authentication  Handler instance 

In [6]:
# create OAuthHandler instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Tweet Stream instantiator

In [ ]:
'''
Run stream and get data. 
NOTE: time error occuring at anypoint in time, will be handled and script wil continue running 
'''


def start_streaming(): 
    while True: 
        try: 
            myStreamListener = MyStreamListener()
            myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
            myStream.filter(languages=["en"], track = word_s_tracked)
            
        except: 
            continue 
        
# NOTE: to stop streaming and close close db , you have to completely shutdown the Kernel
# This is because a while true loop was used above 

start_streaming()
mysqldb.close()

@Joe_Hildebrand You really do talk crap. Here is the lightning strikes that started Victorian and South NSW bushfir… https://t.co/k4gWuWXmlY
So many #Kangaroos in a settlement, not knowing where to go and whre to be safe, as #AustraliaIsBurning .… https://t.co/UbdH7ryfmd
[estimates] 1 billion animals dead
25 human lives lost
20 acres scorched
157 bush/grass fires burning - 56 uncontai… https://t.co/7hpQH5Prs1
New South Wales
This picture breaks my heart #AustraliaOnFire 🔥🌳🔥🐨 😭
US firefighters from California, USA already arrived at Sydney Int Airport, #Australia 🇦🇺

Thy were sending to help… https://t.co/98B46RM4N8
Fuck #AustraliaOnFire https://t.co/EcBYrJHuil
Bless your kind soul 💜💜
New South Wales
This picture breaks my heart #AustraliaOnFire 🔥🌳🔥🐨 😭 https://t.co/b4L1vODWqd
